In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

def create_tensor(file,device):
  load_in = []
  load_out = []

  for line in file:
    load_seq=line[:-1].split(",")
    load_in.append(load_seq[:5])
    load_out.append(load_seq[5])

  load_in = np.array(load_in).astype(int)
  load_out = np.array(load_out).astype(int)

  input_tensor = torch.tensor(load_in,dtype=torch.float32).to(device)
  output_tensor = torch.tensor(load_out,dtype=torch.float32).to(device)
  return (input_tensor,output_tensor)

In [2]:
class SimpleNet(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(SimpleNet, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size,hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x,_ = self.lstm(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        return x

In [3]:
def train(model, input_tensor,output_tensor, loss_fn, optimizer, num_epochs):
  index = 0
  batch = []
  for i in range (18):
    batch.append([input_tensor[index:index+34],output_tensor[index:index+34]])
    index+=34

  for epoch in range(num_epochs):
    train_loss = 0
    for (X,y) in batch:
      X,y = X.to(device),y.to(device)
      output = model(X)
      loss = loss_fn(output.squeeze(), y)

      train_loss +=loss.item()


      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    if (epoch + 1) % 100 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
    train_loss = train_loss / len(input_tensor)
  return train_loss

#For Bin Load

In [ ]:
model = SimpleNet(input_size=5,hidden_size=32,output_size=1).to(device)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

load_file = open("load_values_612.txt","r")
input_tensor,output_tensor = create_tensor(load_file,device)

model.train()
num_epochs = 1500
results = train(model,input_tensor,output_tensor,loss_fn,optimizer,num_epochs)

print("Loss", results)


Epoch [100/1500], Loss: 1.5486
Epoch [200/1500], Loss: 1.5495
Epoch [300/1500], Loss: 1.1728
Epoch [400/1500], Loss: 1.1851


KeyboardInterrupt: ignored

In [ ]:
input_tensor_test = torch.tensor([[15,15,23,31,37]], dtype=torch.float32)
predicted_output = model(input_tensor_test.to(device)).item()
print("Predicted next number:", predicted_output) #43

input_tensor_test = torch.tensor([[30,35,43,49,57]], dtype=torch.float32)
predicted_output = model(input_tensor_test.to(device)).item() #65
print("Predicted next number:", predicted_output)

input_tensor_test = torch.tensor([[44,49,55,63,71]], dtype=torch.float32)
predicted_output = model(input_tensor_test.to(device)).item()
print("Predicted next number:", predicted_output) #76


In [ ]:
from pathlib import Path

MODEL_PATH  = Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)

MODEL_NAME = "bin_load_nn.pth"
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(),
           f=MODEL_SAVE_PATH)

# For temperature

In [13]:
temp_model = SimpleNet(input_size=5,hidden_size=32,output_size=1).to(device)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(temp_model.parameters(), lr=0.001)

load_file = open("temp_values_612.txt","r")
input_tensor,output_tensor = create_tensor(load_file,device)

temp_model.train()
num_epochs = 1500
results = train(temp_model,input_tensor,output_tensor,loss_fn,optimizer,num_epochs)

print("Loss", results)


Epoch [100/1500], Loss: 4.1727
Epoch [200/1500], Loss: 2.8673
Epoch [300/1500], Loss: 2.0382
Epoch [400/1500], Loss: 2.0067
Epoch [500/1500], Loss: 2.0270
Epoch [600/1500], Loss: 2.1252
Epoch [700/1500], Loss: 2.1288
Epoch [800/1500], Loss: 2.1127
Epoch [900/1500], Loss: 2.0883
Epoch [1000/1500], Loss: 2.0302
Epoch [1100/1500], Loss: 1.9889
Epoch [1200/1500], Loss: 1.9362
Epoch [1300/1500], Loss: 1.8735
Epoch [1400/1500], Loss: 1.8142
Epoch [1500/1500], Loss: 1.7788
Loss 0.04139346643990161


In [14]:
input_tensor_test = torch.tensor([[39,41,40,39,39]], dtype=torch.float32)
predicted_output = temp_model(input_tensor_test.to(device)).item()
print("Predicted next number:", predicted_output) #38

input_tensor_test = torch.tensor([[37,38,37,38,38]], dtype=torch.float32)
predicted_output = temp_model(input_tensor_test.to(device)).item() #39
print("Predicted next number:", predicted_output)

input_tensor_test = torch.tensor([[37,36,36,38,38]], dtype=torch.float32)
predicted_output = temp_model(input_tensor_test.to(device)).item()
print("Predicted next number:", predicted_output) #37

Predicted next number: 39.91593551635742
Predicted next number: 37.736412048339844
Predicted next number: 36.277442932128906


In [15]:
from pathlib import Path

MODEL_PATH  = Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)

MODEL_NAME = "bin_temp_nn.pth"
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=temp_model.state_dict(),
           f=MODEL_SAVE_PATH)

Saving model to: models/bin_temp_nn.pth


# For humidity

In [9]:
hum_model = SimpleNet(input_size=5,hidden_size=32,output_size=1).to(device)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(hum_model.parameters(), lr=0.0001)

load_file = open("hum_values_612.txt","r")
input_tensor,output_tensor = create_tensor(load_file,device)

hum_model.train()
num_epochs = 1500
results = train(hum_model,input_tensor,output_tensor,loss_fn,optimizer,num_epochs)

print("Loss", results)

Epoch [100/1500], Loss: 68.7704
Epoch [200/1500], Loss: 19.0892
Epoch [300/1500], Loss: 6.1682
Epoch [400/1500], Loss: 4.5094
Epoch [500/1500], Loss: 4.4172
Epoch [600/1500], Loss: 4.1892
Epoch [700/1500], Loss: 3.7822
Epoch [800/1500], Loss: 3.4363
Epoch [900/1500], Loss: 3.1290
Epoch [1000/1500], Loss: 2.6246
Epoch [1100/1500], Loss: 2.1298
Epoch [1200/1500], Loss: 1.8623
Epoch [1300/1500], Loss: 1.7746
Epoch [1400/1500], Loss: 1.5624
Epoch [1500/1500], Loss: 1.4887
Loss 0.057399201821657565


In [10]:
input_tensor_test = torch.tensor([[30,33,36,39,37]], dtype=torch.float32)
predicted_output = hum_model(input_tensor_test.to(device)).item()
print("Predicted next number:", predicted_output) #41

input_tensor_test = torch.tensor([[33,33,35,38,39]], dtype=torch.float32)
predicted_output = hum_model(input_tensor_test.to(device)).item() #40
print("Predicted next number:", predicted_output)

input_tensor_test = torch.tensor([[40,41,44,30,31]], dtype=torch.float32)
predicted_output = hum_model(input_tensor_test.to(device)).item()
print("Predicted next number:", predicted_output) #32


Predicted next number: 35.48539733886719
Predicted next number: 39.773681640625
Predicted next number: 31.625503540039062


In [12]:
from pathlib import Path

MODEL_PATH  = Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)

MODEL_NAME = "bin_hum_nn.pth"
MODEL_SAVE_PATH = MODEL_PATH/MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=hum_model.state_dict(),
           f=MODEL_SAVE_PATH)

Saving model to: models/bin_hum_nn.pth


In [ ]:
model_2 = SimpleNet(input_size=5,hidden_size=32,output_size=1)
model_2.load_state_dict(torch.load(f="./models/bin_load_nn.pth"))

In [ ]:
input_tensor_test = torch.tensor([[15,15,23,31,37]], dtype=torch.float32)
predicted_output = model_2(input_tensor_test).item()
print("Predicted next number:", predicted_output) #43